In [2]:
!pip install numba

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 11.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import numpy as np
import numba
from numba import jit

# Numba is a just-in-time compiler for Python that translates a subset of Python and NumPy code into fast machine code.

In [ ]:
print(numba.__version__)
# Numba uses Python decorators to transform Python functions into funciton that compile themselves
# most commonly used numba decorators are @jit >> Just-In-Time compiler : which creates a normal funciton for execution on the CPU

0.60.0


In [7]:
# NUmba works best on numerical funcitons that make use of Numoy arrays and functions

@jit(nopython=True)
# nopython=True : forces Numba to compile the function in nopython mode, which means that the function will not use the Python C API
def go_fast(a):
    trace = 0.0
    # assuming square input matrix
    for i in range(a.shape[0]):
        trace+=np.tanh(a[i,i])
        return a+trace

In [16]:
x=np.arange(100).reshape(10,10)
go_fast(x)

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
       [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
       [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
       [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
       [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
       [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]])

In [9]:
go_fast.inspect_types()

go_fast (Array(int64, 2, 'C', False, aligned=True),)
--------------------------------------------------------------------------------
# File: /var/folders/jf/88m_9jxd2h7fz0h5g45ktzv80000gn/T/ipykernel_21901/330467978.py
# --- LINE 3 --- 
# label 0
#   a = arg(0, name=a)  :: array(int64, 2d, C)

@jit(nopython=True)

# --- LINE 4 --- 

# nopython=True : forces Numba to compile the function in nopython mode, which means that the function will not use the Python C API

# --- LINE 5 --- 

def go_fast(a):

    # --- LINE 6 --- 
    #   trace = const(float, 0.0)  :: float64

    trace = 0.0

    # --- LINE 7 --- 

    # assuming square input matrix

    # --- LINE 8 --- 
    #   $6load_global.1 = global(range: <class 'range'>)  :: Function(<class 'range'>)
    #   $10load_attr.3 = getattr(value=a, attr=shape)  :: UniTuple(int64 x 2)
    #   $const12.4 = const(int, 0)  :: Literal[int](0)
    #   $14binary_subscr.5 = static_getitem(value=$10load_attr.3, index=0, index_var=$const12.4, fn=<built-

In [ ]:
go_fast(2*x)
# This first time the function was called, a new version of the function was compiled and executed. If we call it again, the previously generated function executions without another compilation step.
# Numba can also be used to compile functions that are called from other functions

array([[  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.],
       [ 20.,  22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.],
       [ 40.,  42.,  44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.],
       [ 60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,  78.],
       [ 80.,  82.,  84.,  86.,  88.,  90.,  92.,  94.,  96.,  98.],
       [100., 102., 104., 106., 108., 110., 112., 114., 116., 118.],
       [120., 122., 124., 126., 128., 130., 132., 134., 136., 138.],
       [140., 142., 144., 146., 148., 150., 152., 154., 156., 158.],
       [160., 162., 164., 166., 168., 170., 172., 174., 176., 178.],
       [180., 182., 184., 186., 188., 190., 192., 194., 196., 198.]])

In [11]:
# bencmarking the performance of the function
%timeit go_fast(x)

419 ns ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
# testing python version of the function
np.testing.assert_array_equal(go_fast(x), go_fast.py_func(x))

In [13]:
# benchmarking the performance of the python version of the function
%timeit go_fast.py_func(x)

1.55 µs ± 36.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [14]:
# Numpy version
def go_numpy(a):
    return a + np.tanh(np.diagonal(a)).sum()

In [ ]:
np.testing.assert_array_equal(go_numpy(x), go_fast(x))